In [1]:
# API key를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API Key 정보 로드
load_dotenv()

True

In [2]:
# LangSmith 추적 허용
from langchain_teddynote import logging

# projetct name
logging.langsmith("Voice agent")

LangSmith 추적을 시작합니다.
[프로젝트명]
Voice agent


In [3]:
import warnings

warnings.filterwarnings("ignore")

**상담사 역할 정의**
시뮬레이션에서 상담사 역할을 하는 챗봇을 정의합니다.

참고

* call_chatbot 내의 구현은 설정 가능하며, 내부에서 사용한 모델을 Agent 로 변경하는 것도 가능합니다.
* call_chatbot 은 사용자로부터 메시지를 입력으로 받아, 고객을 상담하는 역할을 부여하겠습니다.
* 고객 지원 시나리오에서의 대화 응답 생성에 활용될 수 있습니다.

In [4]:
from langgraph.graph.message import add_messages  
from typing import Annotated  
from typing_extensions import TypedDict  


# State 정의  
class State(TypedDict):  
    messages: Annotated[list, add_messages]  # 사용자 - 상담사 간의 대화 메시지  

## 상담사 역할 정의

In [5]:
from typing import List
from langchain_teddynote.models import LLMs, get_model_name
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder  # 수정: ChatPromptTemplate 추가
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI  # 추가: ChatOpenAI import

# model name (임시 테스트)
MODEL_NAME = "gpt-3.5-turbo"  # get_model_name(LLMs) 대신 직접 지정

def call_chatbot(messages: List[BaseMessage]) -> dict:
    # 프롬프트 템플릿 설정
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You're a system that takes orders from Servoway, Answer in Korean"),
            MessagesPlaceholder(variable_name="messages")
        ]
    )
    # 모델 초기화 (오타 수정)
    model = ChatOpenAI(model=MODEL_NAME, temperature=0.6)
    # 체인 구성
    chain = prompt | model | StrOutputParser()
    # 실행
    return chain.invoke({"messages": messages})  # 수정: "message" → "messages"


In [6]:
call_chatbot(["user", "샌드위치 한 개 주세요"])

Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


'알겠습니다. 어떤 종류의 샌드위치를 주문하시겠습니까?'

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTP

## 고객 역할 정의

In [7]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

def create_scenario(name:str, instructions:str):
    # system 프롬프트를 정의: 필요에 따라 변경
    system_prompt_template = """ 당신은 서보웨이의 고객입니다.
    단일 샌드위치를 주문할 수 있고, 햄, 치즈, 양사추를 3개 제한으로 추가할 수 있습니다.

    [중요]
    - 주문과 관련된 대답만 해야합니다.
    - 한국어로 대화를 해야 합니다.
"""
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt_template),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )

    prompt = prompt.partial(name=name, instructions=instructions)
    return prompt

In [8]:
# 사용자 지시사항을 정의합니다.  
instructions = """당신은 서보웨이 고객입니다.
- 단일 샌드위치를 주문하세요.
- 추가 재료(햄, 치즈, 양상추)는 각 3개까지 선택할 수 있습니다.
- 주문과 관련된 대답만 하세요.
- 모든 대화는 한국어로 하세요."""  

# 사용자 이름을 정의합니다.  
name = "Serbo"  

create_scenario(name, instructions).pretty_print()  

================================ System Message ================================

 당신은 서보웨이의 고객입니다.
    단일 샌드위치를 주문할 수 있고, 햄, 치즈, 양사추를 3개 제한으로 추가할 수 있습니다.

    [중요]
    - 주문과 관련된 대답만 해야합니다.
    - 한국어로 대화를 해야 합니다.


============================= Messages Placeholder =============================

{messages}


In [9]:
# OpenAI chatbot init
model = ChatOpenAI(model=MODEL_NAME, temperature=0.1)

# 시뮬레이션된 사용자 대화를 생성합니다.
simulated_user = create_scenario(name, instructions) | model | StrOutputParser()

In [10]:
from langchain_core.messages import HumanMessage  

# 시뮬레이션된 사용자에게 메시지를 전달  
messages = [HumanMessage(content="안녕하세요? 어떻게 도와 드릴까요?")]  
simulated_user.invoke({"messages": messages})  


'한 개의 샌드위치를 주문하고 싶습니다. 가능한 재료는 무엇인가요?'

## 노드 정의

In [11]:
from langchain_core.messages import AIMessage

# 상담사 역할
def ai_assistant_node(messages):
    # 상담사 응답 호출
    ai_response = call_chatbot(messages)

    #AI 상담사의 응답을 반환
    return {"messages": [("assistant", ai_response)]}

### 상담사 역할의 노드를 호출

In [15]:
ai_assistant_node(  
    [  
        ("user", "안녕하세요?"),  
        ("assistant", "안녕하세요! 어떻게 도와드릴까요?"),  
        ("user", "주문은 어떻게 하나요??"),  
    ]  
)  

{'messages': [('assistant', '주문하실 음식이나 상품을 말씀해주세요. 어떤 것을 주문하시겠어요?')]}